In [5]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas numpy scipy


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


# Load the dataset
file_path = 'C:/Users/USER/Documents/OSUN_election_data.csv' 
df = pd.read_csv(file_path)

df['Address'] = df['PU-Name'] + ', ' + df['Ward'] + ', ' +  df['LGA'] + ', ' +df['State'] + ', Nigeria'

df.to_csv('C:/Users/USER/Documents/OSUN_election_data_with_address.csv', index=False)

df.head()  

,State,LGA,Ward,PU-Code,PU-Name,Accredited_Voters,Registered_Voters,Results_Found,Transcription_Count,Result_Sheet_Stamped,Result_Sheet_Corrected,Result_Sheet_Invalid,Result_Sheet_Unclear,Result_Sheet_Unsigned,APC,LP,PDP,NNPP,Results_File,Address
0,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-001,TOWN HALL IWARA,181,942,True,-1,False,False,False,False,UNKNOWN,44,273,0,0,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"TOWN HALL IWARA, IWARA, ATAKUMOSA EAST, OSUN, ..."
1,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-003,"L.A. SCHOOL, IWIKUN",245,881,True,-1,False,False,False,False,UNKNOWN,40,11,175,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"L.A. SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OS..."
2,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-004,"METHODIST PRY. SCHOOL, AYETORO",117,308,True,-1,False,False,False,False,UNKNOWN,14,3,99,1,https://inec-cvr-cache.s3.eu-west-1.amazonaws....,"METHODIST PRY. SCHOOL, AYETORO, IWARA, ATAKUMO..."
3,OSUN,ATAKUMOSA EAST,IWARA,29-01-01-007,OPEN SPACE ODO-IWARA,187,538,True,-1,False,False,False,False,UNKNOWN,25,4,155,0,https://docs.inecelectionresults.net/elections...,"OPEN SPACE ODO-IWARA, IWARA, ATAKUMOSA EAST, O..."
4,OSUN,ATAKUMOSA EAST,IGANGAN,29-01-02-001,"N.U.D PRY. SCHOL, IGANGAN",187,558,True,-1,False,False,False,False,UNKNOWN,22,0,161,0,https://docs.inecelectionresults.net/elections...,"N.U.D PRY. SCHOL, IGANGAN, IGANGAN, ATAKUMOSA ..."


In [ ]:
import pandas as pd
import requests
import time
import re


api_key = 'a7a94896987848088e4cf2828c1e104a'


def clean_address(address):
 
    address = re.sub(r'[^\w\s,]', '', address)
 
    address = re.sub(r'\s+', ' ', address)
   
    address = address.strip()
    return address

def geocode_address(address, retries=5):
    base_url = "https://api.geoapify.com/v1/geocode/search"
    params = {
        'text': address,
        'apiKey': api_key
    }
    for attempt in range(retries):
        try:
            print(f"Geocoding attempt {attempt + 1} for address: {address}")
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            if data['features']:
                location = data['features'][0]['geometry']['coordinates']
                print(f"Success: {address} -> (Latitude: {location[1]}, Longitude: {location[0]})")
                return location[1], location[0]
            else:
                print(f"No result for address: {address}")
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}. Attempt {attempt + 1}/{retries} failed for address: {address}")
            time.sleep(5)  
    return None, None


file_path = 'C:/Users/USER/Documents/OSUN_election_data_with_address.csv'
df = pd.read_csv(file_path)


df['Cleaned_Address'] = df['Address'].apply(clean_address)

latitudes = []
longitudes = []

for index, row in df.iterrows():
    cleaned_address = row['Cleaned_Address']
    latitude, longitude = geocode_address(cleaned_address)
    latitudes.append(latitude)
    longitudes.append(longitude)

df['Latitude'] = latitudes
df['Longitude'] = longitudes


df.to_csv('C:/Users/USER/Documents/OSUN_election_data_geocoded.csv', index=False)

print(df.head())


Geocoding attempt 1 for address: TOWN HALL IWARA, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
Success: TOWN HALL IWARA, IWARA, ATAKUMOSA EAST, OSUN, Nigeria -> (Latitude: 7.902689, Longitude: 4.313525)
Geocoding attempt 1 for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
No result for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
Geocoding attempt 2 for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
No result for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
Geocoding attempt 3 for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
No result for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
Geocoding attempt 4 for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
No result for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
Geocoding attempt 5 for address: LA SCHOOL, IWIKUN, IWARA, ATAKUMOSA EAST, OSUN, Nigeria
No result for address: LA SCHOOL, IWIKUN, IW

In [ ]:
import pandas as pd

file_path = 'C:/Users/USER/Documents/OSUN_election_data_geocoded.csv'
df = pd.read_csv(file_path)

df_cleaned = df.dropna(subset=['Longitude', 'Latitude'])


print(f"Original dataset size: {df.shape[0]}")
print(f"Cleaned dataset size: {df_cleaned.shape[0]}")


cleaned_file_path = 'C:/Users/USER/Documents/OSUN_election_data_cleaned.csv'
df_cleaned.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved to: {cleaned_file_path}")
print(df_cleaned.head())

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import distance


cleaned_file_path = 'C:/Users/USER/Documents/OSUN_election_data_cleaned.csv'
df = pd.read_csv(cleaned_file_path)


df = df.dropna(subset=['Latitude', 'Longitude'])


def calculate_distances(df, radius=1):
    distances = distance.cdist(df[['Latitude', 'Longitude']], df[['Latitude', 'Longitude']], 'euclidean')
    radius_threshold = radius / 111  # Convert km to degrees
    neighbors = [np.where(distances[i] < radius_threshold)[0].tolist() for i in range(len(df))]
    return neighbors

df['Neighbors'] = calculate_distances(df)


def calculate_outlier_score(df, party):
    outlier_scores = []
    for i, row in df.iterrows():
        neighbors = df.iloc[row['Neighbors']]
        if len(neighbors) > 1:
            mean_votes = neighbors[party].mean()
            std_votes = neighbors[party].std()
            score = (row[party] - mean_votes) / std_votes if std_votes != 0 else 0
            outlier_scores.append(score)
        else:
            outlier_scores.append(0)
    return outlier_scores



In [ ]:

df['APC_Outlier_Score'] = calculate_outlier_score(df, 'APC')
df['PDP_Outlier_Score'] = calculate_outlier_score(df, 'PDP')
df['LP_Outlier_Score'] = calculate_outlier_score(df, 'LP')

top_3_apc = df.nlargest(3, 'APC_Outlier_Score')
top_3_pdp = df.nlargest(3, 'PDP_Outlier_Score')
top_3_lp = df.nlargest(3, 'LP_Outlier_Score')

top_3_apc_df = top_3_apc[['PU-Name', 'Ward', 'LGA', 'State', 'APC', 'APC_Outlier_Score']]
top_3_pdp_df = top_3_pdp[['PU-Name', 'Ward', 'LGA', 'State', 'PDP', 'PDP_Outlier_Score']]
top_3_lp_df = top_3_lp[['PU-Name', 'Ward', 'LGA', 'State', 'LP', 'LP_Outlier_Score']]

print("Top 3 APC Outliers:")
print(top_3_apc_df)
print("\nTop 3 PDP Outliers:")
print(top_3_pdp_df)
print("\nTop 3 LP Outliers:")
print(top_3_lp_df)

In [ ]:
top_3_apc_df.to_csv('top_3_apc_outliers.csv', index=False)
top_3_pdp_df.to_csv('top_3_pdp_outliers.csv', index=False)
top_3_lp_df.to_csv('top_3_lp_outliers.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
top_3_apc = df.nlargest(3, 'APC_Outlier_Score')
top_3_pdp = df.nlargest(3, 'PDP_Outlier_Score')
top_3_lp = df.nlargest(3, 'LP_Outlier_Score')


In [ ]:

plt.figure(figsize=(10, 6))
plt.bar(top_3_apc['PU-Name'], top_3_apc['APC_Outlier_Score'], color='blue')
plt.xlabel('Polling Unit Name')
plt.ylabel('Outlier Score')
plt.title('Top 3 APC Outliers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

plt.figure(figsize=(10, 6))
plt.bar(top_3_pdp['PU-Name'], top_3_pdp['PDP_Outlier_Score'], color='green')
plt.xlabel('Polling Unit Name')
plt.ylabel('Outlier Score')
plt.title('Top 3 PDP Outliers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()




In [3]:
plt.figure(figsize=(10, 6))
plt.bar(top_3_lp['PU-Name'], top_3_lp['LP_Outlier_Score'], color='red')
plt.xlabel('Polling Unit Name')
plt.ylabel('Outlier Score')
plt.title('Top 3 LP Outliers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


NameError: name 'plt' is not defined

In [4]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from haversine import haversine, Unit

# Assuming df contains your dataset with columns 'Latitude', 'Longitude', 'PU-Name', 'Ward', 'LGA', 'State', and outlier scores

# Function to calculate nearest neighbors
def find_nearest_polling_units(df, outlier_df, k=1):
    # Create a BallTree using radians as distance metric
    coords = np.deg2rad(df[['Latitude', 'Longitude']].to_numpy())
    tree = BallTree(coords, metric='haversine')
    
    # Calculate distances and find nearest neighbors for each outlier
    nearest_polling_units = []
    for i, outlier_row in outlier_df.iterrows():
        outlier_coords = np.deg2rad(outlier_row[['Latitude', 'Longitude']])
        dist, ind = tree.query([outlier_coords], k=k+1)  # +1 to exclude self
        nearest_indices = ind[0][1:]  # Exclude the outlier itself
        nearest_units = df.iloc[nearest_indices][['PU-Name', 'Ward', 'LGA', 'State']]
        nearest_polling_units.append(nearest_units.values.tolist())
    
    return nearest_polling_units

# Example usage:
# Replace 'top_3_apc' with your actual DataFrame containing APC outliers
nearest_apc_units = find_nearest_polling_units(df, top_3_apc, k=1)
nearest_pdp_units = find_nearest_polling_units(df, top_3_pdp, k=1)
nearest_lp_units = find_nearest_polling_units(df, top_3_lp, k=1)

# Display or use nearest units as needed
print("Nearest APC Units:")
print(nearest_apc_units)
print("\nNearest PDP Units:")
print(nearest_pdp_units)
print("\nNearest LP Units:")
print(nearest_lp_units)


ModuleNotFoundError: No module named 'haversine'